In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
import seaborn as sns

In [3]:
df = pd.read_csv("Per_100,000Daily.csv")

In [4]:
assignments = pd.read_csv("CA.csv")

In [5]:
df.columns

Index(['Unnamed: 0', 'Wave 3_Patients Admitted Due to COVID',
       'Wave 3_Patients Admitted Not Due to COVID',
       'Wave 3_Patients Newly Admitted', 'Wave 3_Patients Discharged',
       'Wave 3_Patients Currently in ICU',
       'Wave 3_Patients Currently ICU Intubated', 'Wave 3_Patients Expired'],
      dtype='object')

In [6]:
assignments.columns

Index(['Unnamed: 0', 'cluster'], dtype='object')

In [7]:
assdf = df.merge(assignments, left_on="Unnamed: 0", right_on='Unnamed: 0')

In [8]:
#U.S. Census Bureau. (2021). INCOME IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS). American Community Survey, ACS 5-Year Estimates Subject Tables, Table S1901. Retrieved April 1, 2025, from https://data.census.gov/table/ACSST5Y2021.S1901?t=Income+and+Poverty&g=040XX00US36$0500000&y=2021&tp=true.
Income = pd.read_csv("ACSST5Y2021.S1901-2025-04-02T010000.csv")
#U.S. Census Bureau. "EDUCATIONAL ATTAINMENT." American Community Survey, ACS 5-Year Estimates Subject Tables, Table S1501, 2021, https://data.census.gov/table/ACSST5Y2021.S1501?t=Education:Educational+Attainment&g=040XX00US36$0500000&y=2021&tp=true. Accessed on April 1, 2025.
education = pd.read_csv("ACSST5Y2021.S1501-2025-04-02T010231.csv")
#U.S. Census Bureau. (2021). OCCUPATION BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER. American Community Survey, ACS 5-Year Estimates Subject Tables, Table S2406. Retrieved April 1, 2025, from https://data.census.gov/table/ACSST5Y2021.S2406?t=Class+of+Worker:Employment:Employment+and+Labor+Force+Status:Occupation&g=040XX00US36$0500000&y=2021&tp=true.
workerclass = pd.read_csv("ACSST5Y2021.S2406-2025-04-02T010549.csv")
#U.S. Census Bureau. (2021). COMPARATIVE ECONOMIC CHARACTERISTICS. American Community Survey, ACS 5-Year Estimates Comparison Profiles, Table CP03. Retrieved April 1, 2025, from https://data.census.gov/table/ACSCP5Y2021.CP03?t=Class+of+Worker:Employment:Employment+and+Labor+Force+Status:Occupation&g=040XX00US36$0500000&y=2021&tp=true.
unemployment = pd.read_csv("ACSCP5Y2021.CP03-2025-04-02T010708.csv")

In [9]:
def bestfunction(newstring):
    templist = newstring.split(",")
    if len(templist) == 2:
        tempstring = templist[0] + templist[1]
        newstring = int(tempstring)
    elif len(templist) == 3:
        tempstring = templist[0] + templist[1] + templist[2]
        newstring = int(tempstring)
    else:
        newstring = int(newstring)
    return newstring

In [10]:
newdf = pd.DataFrame(columns=["County","Median Income"])

In [11]:
bestlist = []
u = 1
for x in range(0,806):
    if u == 1:
        bestlist.append(Income.iloc[x,0])
        u = u + 1
    elif u == 3:
        bestlist.append(bestfunction(Income.iloc[x,12]))
        u = u + 1
    elif u == 13:
        newdf.loc[len(newdf)] = [bestlist[0], bestlist[1]]
        bestlist = []
        u = 1

    else:
        u = u + 1

In [12]:
df1list = df["Unnamed: 0"].unique()

In [13]:
for index,row in newdf.iterrows():
    newstring = newdf.iloc[index,0]
    wordlist = newstring.split(" ")
    if len(wordlist) > 4:
        beststring = wordlist[0] +  " " + wordlist[1]
        newdf.iloc[index,0] = beststring.upper()
    else:
        newdf.iloc[index,0] = wordlist[0].upper()

In [14]:
df2list = newdf["County"].unique()

In [15]:
for x in df1list:
    if x in df2list:
        pass
    else:
        print(x)

In [16]:
assdf = assdf.merge(newdf, left_on="Unnamed: 0", right_on="County")
assdf = assdf.drop(columns="County")

In [17]:
newdf = pd.DataFrame(columns=["County","Less than high school","High school graduate","Some college or associate's degree","Bachelor's degree or higher"])

In [18]:
bestlist = []
u = 1
for x in range(0,1178):
    if u == 1:
        bestlist.append(education.iloc[x,0])
        u = u + 1
    elif u == 3:
        totalpop = bestfunction(education.iloc[x,2]) + bestfunction(education.iloc[x,7])
        lesspop = bestfunction(education.iloc[x,3]) + bestfunction(education.iloc[x,8]) + bestfunction(education.iloc[x,9])
        highpop = bestfunction(education.iloc[x,4]) + bestfunction(education.iloc[x,10])
        somepop = bestfunction(education.iloc[x,5]) + bestfunction(education.iloc[x,11]) + bestfunction(education.iloc[x,12])
        higherpop = bestfunction(education.iloc[x,6]) + bestfunction(education.iloc[x,13]) + bestfunction(education.iloc[x,14])
        bestlist.append(lesspop/totalpop)
        bestlist.append(highpop/totalpop)
        bestlist.append(somepop/totalpop)
        bestlist.append(higherpop/totalpop)
        if bestlist[0] in ['Kings County, New York','New York County, New York','Queens County, New York']:
            print(totalpop)
            print(lesspop)
            print(highpop)
            print(somepop)
            print(higherpop)
        u = u + 1
    elif u == 19:
        newdf.loc[len(newdf)] = [bestlist[0], bestlist[1],bestlist[2],bestlist[3],bestlist[4]]
        bestlist = []
        u = 1
    else:
        u = u + 1

2090262
332551
532622
441078
784011
1427229
159341
186076
230721
851091
1908997
319034
507104
463991
618868


In [19]:
for index,row in newdf.iterrows():
    newstring = newdf.iloc[index,0]
    wordlist = newstring.split(" ")
    if len(wordlist) > 4:
        beststring = wordlist[0] +  " " + wordlist[1]
        newdf.iloc[index,0] = beststring.upper()
    else:
        newdf.iloc[index,0] = wordlist[0].upper()

In [20]:
assdf = assdf.merge(newdf, left_on="Unnamed: 0", right_on="County")
assdf = assdf.drop(columns="County")

In [21]:
newdf = pd.DataFrame(columns=["County","Management, business, science, and arts occupations","Service occupations","Sales and office occupations","Natural resources, construction, and maintenance occupations","Production, transportation, and material moving occupations"])

In [22]:
bestlist = []
u = 1
for x in range(0,1178):
    if u == 1:
        bestlist.append(workerclass.iloc[x,0])
        u = u + 1
    elif u == 3:
        totalpop = bestfunction(workerclass.iloc[x,1])
        lesspop = bestfunction(workerclass.iloc[x,2])
        highpop = bestfunction(workerclass.iloc[x,3])
        somepop = bestfunction(workerclass.iloc[x,4])
        higherpop = bestfunction(workerclass.iloc[x,5])
        temppop = bestfunction(workerclass.iloc[x,6])
        bestlist.append(lesspop/totalpop)
        bestlist.append(highpop/totalpop)
        bestlist.append(somepop/totalpop)
        bestlist.append(higherpop/totalpop)
        bestlist.append(temppop/totalpop)
        u = u + 1
    elif u == 19:
        newdf.loc[len(newdf)] = [bestlist[0], bestlist[1],bestlist[2],bestlist[3],bestlist[4],bestlist[5]]
        bestlist = []
        u = 1
    else:
        u = u + 1

In [23]:
for index,row in newdf.iterrows():
    newstring = newdf.iloc[index,0]
    wordlist = newstring.split(" ")
    if len(wordlist) > 4:
        beststring = wordlist[0] +  " " + wordlist[1]
        newdf.iloc[index,0] = beststring.upper()
    else:
        newdf.iloc[index,0] = wordlist[0].upper()

In [24]:
assdf = assdf.merge(newdf, left_on="Unnamed: 0", right_on="County")
assdf = assdf.drop(columns="County")

In [25]:
newdf = pd.DataFrame(columns=["County","Unemployed","POVERTY"])

In [26]:
def secondbestfunction(newstring):
    newstring = newstring.replace("%","")
    newstring = float(newstring)
    return newstring

In [27]:
bestlist = []
u = 1
for x in range(0,248):
    if u == 1:
        bestlist.append(unemployment.iloc[x,0])
        u = u + 1
    elif u == 2:
        totalpop = secondbestfunction(unemployment.iloc[x,6])
        lesspop = secondbestfunction(unemployment.iloc[x,10])
        bestlist.append(totalpop)
        bestlist.append(lesspop)
        u = u + 1
    elif u == 4:
        newdf.loc[len(newdf)] = [bestlist[0], bestlist[1],bestlist[2]]
        bestlist = []
        u = 1
    else:
        u = u + 1

In [28]:
for index,row in newdf.iterrows():
    newstring = newdf.iloc[index,0]
    wordlist = newstring.split(" ")
    if len(wordlist) > 4:
        beststring = wordlist[0] +  " " + wordlist[1]
        newdf.iloc[index,0] = beststring.upper()
    else:
        newdf.iloc[index,0] = wordlist[0].upper()

In [29]:
assdf = assdf.merge(newdf, left_on="Unnamed: 0", right_on="County")
assdf = assdf.drop(columns="County")

In [30]:
df1 = assdf[assdf["cluster"] == 1]
df2 = assdf[assdf["cluster"] == 2]
df3 = assdf[assdf["cluster"] == 3]

In [31]:
#linear regresion
#decision trees
#random forest

In [32]:
from scipy.stats import pearsonr

In [33]:
testlist = ['Wave 3_Patients Admitted Due to COVID',
       'Wave 3_Patients Admitted Not Due to COVID',
       'Wave 3_Patients Newly Admitted', 'Wave 3_Patients Discharged',
       'Wave 3_Patients Currently in ICU',
       'Wave 3_Patients Currently ICU Intubated', 'Wave 3_Patients Expired']

In [34]:
trainlist = ['Median Income', 'Less than high school',
       'High school graduate', "Some college or associate's degree",
       "Bachelor's degree or higher",
       'Management, business, science, and arts occupations',
       'Service occupations', 'Sales and office occupations',
       'Natural resources, construction, and maintenance occupations',
       'Production, transportation, and material moving occupations',
       'Unemployed', 'POVERTY']

In [35]:
for x in testlist:
    print(x)
    print("")
    for y in trainlist:
        correlation_coefficient, p_value = pearsonr(df1[x], df1[y])

        print(f"Pearson correlation coefficient: {correlation_coefficient}")
        print(f"P-value: {p_value}")

        # Interpret the p-value
        alpha = 0.10  # Significance level
        if p_value < alpha:
            print(f"The correlation for {x} & {y} is statistically significant.")
            print(1)
        else:
            print(f"The correlation for {x} & {y} is not statistically significant.")

Wave 3_Patients Admitted Due to COVID

Pearson correlation coefficient: 0.35272981428597144
P-value: 0.23715808776899644
The correlation for Wave 3_Patients Admitted Due to COVID & Median Income is not statistically significant.
Pearson correlation coefficient: -0.49914354686305834
P-value: 0.08247289024188295
The correlation for Wave 3_Patients Admitted Due to COVID & Less than high school is statistically significant.
1
Pearson correlation coefficient: -0.37628577777006544
P-value: 0.20507500550902488
The correlation for Wave 3_Patients Admitted Due to COVID & High school graduate is not statistically significant.
Pearson correlation coefficient: 0.3253679464825892
P-value: 0.2780143603525807
The correlation for Wave 3_Patients Admitted Due to COVID & Some college or associate's degree is not statistically significant.
Pearson correlation coefficient: 0.3056299871942433
P-value: 0.30986782197805635
The correlation for Wave 3_Patients Admitted Due to COVID & Bachelor's degree or highe

In [36]:
for x in testlist:
    print(x)
    print("")
    for y in trainlist:
        correlation_coefficient, p_value = pearsonr(df2[x], df2[y])

        print(f"Pearson correlation coefficient: {correlation_coefficient}")
        print(f"P-value: {p_value}")

        # Interpret the p-value
        alpha = 0.10  # Significance level
        if p_value < alpha:
            print(f"The correlation for {x} & {y} is statistically significant.")
            print(1)
        else:
            print(f"The correlation for {x} & {y} is not statistically significant.")

Wave 3_Patients Admitted Due to COVID

Pearson correlation coefficient: 0.08180809145826934
P-value: 0.7316924690861287
The correlation for Wave 3_Patients Admitted Due to COVID & Median Income is not statistically significant.
Pearson correlation coefficient: -0.4508788927948958
P-value: 0.0460180073260142
The correlation for Wave 3_Patients Admitted Due to COVID & Less than high school is statistically significant.
1
Pearson correlation coefficient: -0.32031504311174447
P-value: 0.16855697447893292
The correlation for Wave 3_Patients Admitted Due to COVID & High school graduate is not statistically significant.
Pearson correlation coefficient: -0.09870295907919172
P-value: 0.6788710119736883
The correlation for Wave 3_Patients Admitted Due to COVID & Some college or associate's degree is not statistically significant.
Pearson correlation coefficient: 0.39682245313225994
P-value: 0.0832153459747203
The correlation for Wave 3_Patients Admitted Due to COVID & Bachelor's degree or higher

In [37]:
for x in testlist:
    print(x)
    print("")
    for y in trainlist:
        correlation_coefficient, p_value = pearsonr(df3[x], df3[y])

        print(f"Pearson correlation coefficient: {correlation_coefficient}")
        print(f"P-value: {p_value}")

        # Interpret the p-value
        alpha = 0.10  # Significance level
        if p_value < alpha:
            print(f"The correlation for {x} & {y} is statistically significant.")
            print(1)
        else:
            print(f"The correlation for {x} & {y} is not statistically significant.")

Wave 3_Patients Admitted Due to COVID

Pearson correlation coefficient: 0.3133771213547917
P-value: 0.13592783516731594
The correlation for Wave 3_Patients Admitted Due to COVID & Median Income is not statistically significant.
Pearson correlation coefficient: -0.08202065337068057
P-value: 0.7031958081450727
The correlation for Wave 3_Patients Admitted Due to COVID & Less than high school is not statistically significant.
Pearson correlation coefficient: -0.1323467068389953
P-value: 0.5375822539892683
The correlation for Wave 3_Patients Admitted Due to COVID & High school graduate is not statistically significant.
Pearson correlation coefficient: 0.04189122270141274
P-value: 0.8458991139080849
The correlation for Wave 3_Patients Admitted Due to COVID & Some college or associate's degree is not statistically significant.
Pearson correlation coefficient: 0.11241396218022719
P-value: 0.600990095092798
The correlation for Wave 3_Patients Admitted Due to COVID & Bachelor's degree or higher 

In [38]:
df1.to_csv("Cluster1FinalData.csv")
df2.to_csv("Cluster2FinalData.csv")
df3.to_csv("Cluster3FinalData.csv")

In [39]:
from scipy.stats import anderson

In [40]:
def get_range(data):
  return max(data) - min(data)

In [50]:
def testfunc(testthing,data,yesgraph=True):
    if yesgraph == True:
        plt.hist(data[testthing], bins=7)
        plt.show()
    result = anderson(data[testthing])
    print('Test Statistic:', result.statistic)
    print('Critical Values:', result.critical_values)
    print('Significance Levels:', result.significance_level)
    for i in range(len(result.critical_values)):
        sl, cv = result.significance_level[i], result.critical_values[i]
        if result.statistic < cv:
            print('Data looks {} normal (significance level {}%)'.format('likely' if sl > 5 else 'very likely', sl))
        else:
            print('Data does not look normal (significance level {}%)'.format(sl))
    print(f"Mean: {data[testthing].mean()}")
    print(f"Median: {data[testthing].median()}")
    print(f"Range: {get_range(data[testthing])}")

In [42]:
dataset = [assdf, df1, df2, df3]

In [43]:
assdf.columns

Index(['Unnamed: 0', 'Wave 3_Patients Admitted Due to COVID',
       'Wave 3_Patients Admitted Not Due to COVID',
       'Wave 3_Patients Newly Admitted', 'Wave 3_Patients Discharged',
       'Wave 3_Patients Currently in ICU',
       'Wave 3_Patients Currently ICU Intubated', 'Wave 3_Patients Expired',
       'cluster', 'Median Income', 'Less than high school',
       'High school graduate', 'Some college or associate's degree',
       'Bachelor's degree or higher',
       'Management, business, science, and arts occupations',
       'Service occupations', 'Sales and office occupations',
       'Natural resources, construction, and maintenance occupations',
       'Production, transportation, and material moving occupations',
       'Unemployed', 'POVERTY'],
      dtype='object')

In [61]:
for x in dataset:
    testfunc('POVERTY', x,yesgraph = False)

Test Statistic: 0.6878352950390223
Critical Values: [0.542 0.617 0.741 0.864 1.028]
Significance Levels: [15.  10.   5.   2.5  1. ]
Data does not look normal (significance level 15.0%)
Data does not look normal (significance level 10.0%)
Data looks very likely normal (significance level 5.0%)
Data looks very likely normal (significance level 2.5%)
Data looks very likely normal (significance level 1.0%)
Mean: 5.6192982456140355
Median: 5.4
Range: 7.999999999999999
Test Statistic: 0.17931304068262044
Critical Values: [0.497 0.566 0.679 0.792 0.942]
Significance Levels: [15.  10.   5.   2.5  1. ]
Data looks likely normal (significance level 15.0%)
Data looks likely normal (significance level 10.0%)
Data looks very likely normal (significance level 5.0%)
Data looks very likely normal (significance level 2.5%)
Data looks very likely normal (significance level 1.0%)
Mean: 5.2846153846153845
Median: 5.4
Range: 3.8999999999999995
Test Statistic: 1.1566395986227853
Critical Values: [0.506 0.577

In [40]:
assdf

,Unnamed: 0,Wave 3_Patients Admitted Due to COVID,Wave 3_Patients Admitted Not Due to COVID,Wave 3_Patients Newly Admitted,Wave 3_Patients Discharged,Wave 3_Patients Currently in ICU,Wave 3_Patients Currently ICU Intubated,Wave 3_Patients Expired,cluster,Median Income,...,High school graduate,Some college or associate's degree,Bachelor's degree or higher,"Management, business, science, and arts occupations",Service occupations,Sales and office occupations,"Natural resources, construction, and maintenance occupations","Production, transportation, and material moving occupations",Unemployed,POVERTY
0,ALBANY,15.801276,5.404243,4.730517,4.343125,6.684322,2.952363,0.510107,2,73810,...,0.229183,0.293259,0.405160,0.485468,0.162839,0.223278,0.047515,0.080900,3.2,4.9
1,ALLEGANY,6.164191,0.962136,3.326706,3.767005,4.305149,0.423992,0.407685,3,54375,...,0.330523,0.400151,0.188790,0.349464,0.191168,0.188764,0.103841,0.166762,3.6,6.7
2,BRONX,8.309570,7.636697,2.064403,2.761454,3.543901,1.344203,0.247955,2,43726,...,0.279206,0.276540,0.195161,0.269325,0.338137,0.207589,0.062415,0.122533,6.6,11.2
3,BROOME,9.074910,10.451398,1.803543,5.460198,4.735730,2.829237,0.449933,2,53982,...,0.300806,0.345614,0.264026,0.402323,0.200060,0.210401,0.063344,0.123871,4.2,7.5
4,CATTARAUGUS,9.567784,4.415144,4.159479,4.690476,5.349306,4.523310,0.845662,2,53537,...,0.398067,0.309148,0.184619,0.318223,0.198814,0.198239,0.110664,0.174060,3.4,6.0
5,CAYUGA,5.186425,0.824661,2.354756,2.036814,1.371124,0.248392,0.218585,1,59602,...,0.325200,0.340643,0.207692,0.355411,0.193302,0.193675,0.104176,0.153435,2.6,4.5
6,CHAUTAUQUA,6.367679,2.818870,2.812936,2.551819,3.685302,1.661650,0.522233,3,50408,...,0.342276,0.335832,0.212906,0.329995,0.199419,0.193522,0.097187,0.179877,3.0,5.4
7,CHEMUNG,7.526422,6.266016,4.123327,3.367083,6.662144,2.466794,0.765246,2,58175,...,0.347206,0.318125,0.222867,0.346874,0.188080,0.229000,0.075578,0.160468,3.5,6.1
8,CHENANGO,1.925224,0.481306,1.203265,1.893137,2.518835,0.721959,0.128348,1,55690,...,0.413009,0.294282,0.174116,0.322713,0.165240,0.189240,0.111719,0.211088,4.2,7.0
9,CLINTON,10.218918,10.911565,1.707897,3.074215,5.228063,3.131145,0.550322,2,62470,...,0.348376,0.303400,0.230762,0.342979,0.210082,0.207613,0.083539,0.155786,2.4,4.3
